## Table of Contents
### 1. Matrix Factorization
* 기본 아이디어
* user/item bias
* Algorithm

### 2. Training_Test Simultaneously

### 3. Parameter Tuning
* optimal K
* optimal # of Iterations
---

## Matrix Factorization

### 기본 아이디어
user와 item의 관계에 있어서, 둘을 이어주는 잠재 요인(**latent factor**)가 있을 것이다. 즉,

* item: 각각의 item들은 여러 특성을 가지고 있고, 그 특성은 몇개의 **latent factor**로 표현될 수 있다.
* user: 각각의 user들은 그들의 취향을 가지고 있고, 그 취향은 몇개의 **latent factor**로 설명될 수 있다.

따라서, **Matrix Factorization**은 그 둘을 잇는 latent factor와 P(user - latent factor relation), Q(item - latent factor의 relation)을 찾는 과정이다(물론 여느 알고리즘처럼 각각의 latent factor가 무엇을 의미하는 지는 알 수 없다).

### user/item bias
추가로, 마치 Collaborative Filtering에서 user-bias를 고려하였듯이, 예상 평점을 추측할 때 user와 item의 경향성을 함께 고려한다. 따라서, 예상 평점을 구하는 식은 다음과 같다.

``예상 평점 = 전체 평균(b) + user 경향성(b_u) + item 경향성(b_d) + **취향**(P & Q)``

### Algorithm
1. K(# of latent factors)를 정한다. (후에 parameter tuning을 통해 최적 값을 찾는 과정을 거친다)
2. 주어진 K에 따라 행렬 P(m x k), Q(n x k)를 초기화한다.
3. 주어진 P, Q를 이용해 예측 평점 $\hat{R}$를 구한다. ($\matrix{P}$ x $\mathbf{Q}^\top$ = $\hat{R}$)
4. $\matrix{R}$과 $\hat{R}$을 비교해 Error를 구하고, 이를 이용해 P와 Q를 update한다(like using SGD)
5. 전체오차가 미리 정해진 기준 값 이하가 되거나 지정된 iteration 횟수에 도달할 때 까지 3~4를 반복한다.

In [1]:
import pandas as pd
import numpy as np

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jisujung/Desktop/dev/RecSys/python을 이용한 개인화 추천시스템/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings= ratings[['user_id', 'movie_id', 'rating']].astype(int)

class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R= np.array(ratings)
        self.num_users, self.num_items= np.shape(self.R)
        self.K= K
        self.alpha= alpha
        self.beta= beta
        self.iterations= iterations
        self.verbose= verbose

    # RMSE 계산
    def rmse(self):
        x_idx, y_idx= self.R.nonzero()
        self.predictions= []
        self.errors= []
        for x, y in zip(x_idx, y_idx):
            prediction= self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions= np.array(self.predictions)
        self.errors= np.array(self.errors)

        return np.sqrt(np.mean(self.errors ** 2))

    def train(self):
        # Initializing user-feature and movie feature matrix (P, Q)
        # 잘 보면 Q는 (item x feature)로 정의되어있는데, 책에서는 matrix Q는 코드랑 같게 되어있는데 element q가 Q의 element가 아니라 Q.T의 element인 것 처럼 정해지gim
        self.P= np.random.normal(scale= 1./self.K, size=(self.num_users, self.K))
        self.Q= np.random.normal(scale= 1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u= np.zeros(self.num_users)
        self.b_d= np.zeros(self.num_items)
        self.b= np.mean(self.R[self.R.nonzero()])

        # List of training samples - train only nonzero values
        rows, columns= self.R.nonzero()
        self.samples= [(row, column, self.R[row, column]) for (row, column) in zip(rows, columns)]

        # Stochastic GD for given number of iterations
        training_process= []
        for i in range(self.iterations):
            # SGD는 시작점이 중요하므로 shuffle을 통해 순서를 바꿔준다.
            np.random.shuffle(self.samples)
            self.sgd()
            rmse= self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: {}, RMSE: {}".format(i+1, rmse))

    def get_prediction(self, x, y):
        # R에는 zero(null) value가 있을지라도, b, b_u, b_d, P, Q는 그렇지 않다.
        prediction= self.b + self.b_u[x] + self.b_d[y] + self.P[x, :].dot(self.Q[y, :].T)
        return prediction

    def sgd(self):
        for i, j, r in self.samples:
            prediction= self.get_prediction(i, j)
            e= (r - prediction)     
            self.b_u[i]+= self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j]+= self.alpha * (e - self.beta * self.b_d[j])

            # error앞에 2 어디?
            # 앞서 얘기했듯이, 책의 q가 잘못 서술되어있음. 책의 q는 Q의 element가 아니라 Q.T의 element임. w
            self.P[i, :]+= self.alpha * (e*self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :]+= self.alpha * (e*self.P[i, :] - self.beta * self.Q[j, :])


# execute
R_temp= ratings.pivot(index= 'user_id', columns= 'movie_id', values= 'rating').fillna(0)
mf= MF(R_temp, 30, 0.001, 0.02, 100)
train_process= mf.train()

Iteration: 10, RMSE: 0.9585227481580039
Iteration: 20, RMSE: 0.9373806501000475
Iteration: 30, RMSE: 0.9281088106057349
Iteration: 40, RMSE: 0.9225966253787458
Iteration: 50, RMSE: 0.9185301934254234
Iteration: 60, RMSE: 0.914779377630549
Iteration: 70, RMSE: 0.9104629019332259
Iteration: 80, RMSE: 0.9046382388563874
Iteration: 90, RMSE: 0.8962972621456214
Iteration: 100, RMSE: 0.8848484352881525


---
## Training_Test Simultaneously

Train/Test set을 분리하고,training을 진행하면서 중간중간 test set으로 성과를 측정한다.

대부분의 경우, user/item의 id와 DataFrame(or Array)의 index가 일치하지 않는 경우가 많다. 이 경우, 나중에 id로 index를 조회할 때(혹은 그 역의 경우) 제대로 작동 할 수 없으므로, 미리 **실제 아이디와 내부 인덱스를 매핑**해주어야 한다.

In [2]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jisujung/Desktop/dev/RecSys/python을 이용한 개인화 추천시스템/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings= ratings[['user_id', 'movie_id', 'rating']].astype(int)

# Train-Test-Split
from sklearn.utils import shuffle
TRAIN_SIZE= 0.75
ratings= shuffle(ratings, random_state= 1)
cutoff= int(TRAIN_SIZE * len(ratings))
# pivot하기 전에 shuffle! 생각해봐... pivot한 형태면 어떻게 쪼개....
ratings_train= ratings.iloc[:cutoff]
ratings_test= ratings.iloc[cutoff:]

# New MF class for training & testing
# user_id, item_id와 dataframe(or array)의 index를 mapping해야 함!
class New_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R= np.array(ratings)

        item_id_index= []
        index_item_id= []
        for idx, one_id in enumerate(ratings):
            item_id_index.append([one_id, idx])
            index_item_id.append([idx, one_id])
        self.item_id_index= dict(item_id_index)
        self.index_item_id= dict(index_item_id)

        user_id_index= []
        index_user_id= []
        for idx, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, idx])
            index_user_id.append([idx, one_id])
        self.user_id_index= dict(user_id_index)
        self.index_user_id= dict(index_user_id)

        self.num_users, self.num_items= np.shape(self.R)
        self.K= K
        self.alpha= alpha
        self.beta= beta
        self.iterations= iterations
        self.verbose= verbose

    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # SGD to get Optimized b_u, b_d, P, Q
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Test Set
    def set_test(self, ratings_test):
        test_set= []
        for i in range(len(ratings_test)):
            # x, y는 각각 user와 item의 index
            # (i, j, value)의 tuple을 list에 담음으로서 이전 코드와의 통일성을 유지 할 수 있음.
            x= self.user_id_index[ratings_test.iloc[i, 0]]
            y= self.item_id_index[ratings_test.iloc[i, 1]]
            z= ratings_test.iloc[i, 2]
            test_set.append([x, y, z])
            # 0으로 바꿔주면 train시 이 부분 건너 뜀
            self.R[x, y]= 0
        self.test_set= test_set

        return test_set

    def test_rmse(self):
        error= 0
        for one_set in self.test_set:
            predicted= self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)

        return np.sqrt(error/len(self.test_set))

    def test(self):
        # Initializing P, Q, b_u, b_d
        self.P= np.random.normal(scale= 1./self.K, size=(self.num_users, self.K))
        self.Q= np.random.normal(scale= 1./self.K, size=(self.num_items, self.K))
        self.b_u= np.zeros(self.num_users)
        self.b_d= np.zeros(self.num_items)
        self.b= np.mean(self.R[self.R.nonzero()])
        
        row, columns= self.R.nonzero()
        self.samples= [(i, j, self.R[i, j]) for (i, j) in zip(row, columns)]
        
        # run SGD
        training_process= []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1= self.rmse()
            rmse2= self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iterations: %d, Training RMSE: %.4f, Test RMSE: %.4f" % (i+1, rmse1, rmse2))
        
        return training_process
    
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
        
    def full_prediction(self):    
        return self.b + self.b_u[:, np.newaxis], self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)
    

R_temp= ratings.pivot(index= 'user_id', columns= 'movie_id', values= 'rating').fillna(0)
mf= New_MF(R_temp, K=30, alpha=0.001, beta=0.02, iterations=100, verbose=True)
test_set=  mf.set_test(ratings_test)
result= mf.test()

print('')
print(mf.get_one_prediction(1, 2))

Iterations: 10, Training RMSE: 0.9659, Test RMSE: 0.9833
Iterations: 20, Training RMSE: 0.9410, Test RMSE: 0.9644
Iterations: 30, Training RMSE: 0.9298, Test RMSE: 0.9566
Iterations: 40, Training RMSE: 0.9231, Test RMSE: 0.9523
Iterations: 50, Training RMSE: 0.9183, Test RMSE: 0.9496
Iterations: 60, Training RMSE: 0.9145, Test RMSE: 0.9478
Iterations: 70, Training RMSE: 0.9109, Test RMSE: 0.9465
Iterations: 80, Training RMSE: 0.9071, Test RMSE: 0.9453
Iterations: 90, Training RMSE: 0.9025, Test RMSE: 0.9440
Iterations: 100, Training RMSE: 0.8967, Test RMSE: 0.9425

3.3794437988052235


---
## Parameter Tuning
(실제로 코드를 돌려보기에는 조금 무리가 있는 연산량이므로 코드만 짜고 출력은 하지 않음)

이번 예시에서는 최적의 K와 iteration 횟수를 찾아본다.

### K
K는 latent factor의 수를 의미하므로, K가 크면 클수록 다양한 패턴으로 평가 데이터를 설명 할 수 있지만, Training set에 대해 overfitting될 가능성이 커진다.

따라서, Test set의 정확도는 k가 증가함에 따라 같이 증가하였다가 어느 순간 감소할 것이다.

#### How to Find Optimal K
최적의 K를 찾기 위해, 50~260까지의 넓은 범위에 대해 10 간격으로 정확도(RMSE)를 계산한다(그 후 다시 1간격으로 더 정확한 optimal k를 찾을 수 있겠지만, 여기서는 생략한다).

In [ ]:
results= []
index= []
R_temp= ratings.pivot(index= 'user_id', columns= 'movie_id', values= 'rating').fillna(0)
for k in range(50, 261, 10):
    mf= New_MF(R_temp, K= k, alpha= 0.001, beta= 0.02, iterations= 300)
    test_set= mf.set_test(ratings_test)
    result= mf.test()[2]
    index.append(k)
    results.append(result)
    
opt_idx= np.argmax(results)
opt_k= index[opt_idx]

### Iterations
Iterations는 trainig 횟수를 의미하므로, Iterations가 크면 클수록 학습이 더 정확하게 이루어지지만, Training set에 대해 overfitting될 가능성이 커진다. 따라서 Test set의 정확도는 iterations가 증가함에 따라 같이 증가하였다가 어느 순간 감소할 것이다.

#### How to Find Optimal Iterations
학습과정에서 iterations에 충분히 큰 숫자를 주고, RMSE의 변화 양상을 관찰하여 주어진 K에 대한 최적의 iteration 값을 구한다. 이 때, K는 위에서 구한 optimal value를 사용한다.

가장 정확하게 optimal한 (k, iterations)의 조합을 구하기 위해서는 위에서 찾은 K로 고정하는 것이 아니라 다양한 조합을 시도해보아야 하지만, 이 경우 연산량이 폭발적으로 증가하므로 보통 이와 같이 순차적으로 최적의 parameter를 구해나간다.